# Fit Autogluon Multimodal SCF

Use the Autogluon AutoML library to predict ratings using both tabular data and earnings call transcripts.

In [1]:
# Packages
import pandas as pd
import warnings
import os
import numpy as np
warnings.filterwarnings('ignore')
from autogluon.multimodal import MultiModalPredictor

In [2]:
# Set seed
np.random.seed(222)

## Load Data

In [3]:
# list of files in '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
file_list = [f for f in os.listdir(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates') if f.endswith('.parquet')]
# read in all parquet files
df = pd.concat([pd.read_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/' + f) for f in file_list])
print('dataframe')
print(df)

dataframe
    ticker fixed_quarter_date earnings_call_date Rating rating_date  \
0      NEE         2013-01-01         2012-10-24    BBB  2012-09-14   
1      NEE         2013-04-01         2013-01-29    BBB  2013-03-22   
2      NEE         2013-07-01         2013-04-30    BBB  2013-03-22   
3      NEE         2013-10-01         2013-07-30    BBB  2013-03-22   
4      NEE         2014-01-01         2013-11-01    BBB  2013-03-22   
..     ...                ...                ...    ...         ...   
830     KW         2013-10-01         2013-08-07     BB  2013-03-28   
831     KW         2014-01-01         2013-11-06     BB  2013-03-28   
832     KW         2014-04-01         2014-02-27     BB  2014-03-19   
833     KW         2014-07-01         2014-05-09     BB  2014-03-19   
834     KW         2014-10-01         2014-08-08     BB  2014-03-19   

     Rating Rank AAA is 10 Next Rating Next Rating Date Previous Rating  \
0                        7         BBB       2013-03-22       

In [4]:
# Print out column names
print('column names')
for col in df.columns:
    print(col)

column names
ticker
fixed_quarter_date
earnings_call_date
Rating
rating_date
Rating Rank AAA is 10
Next Rating
Next Rating Date
Previous Rating
Previous Rating Date
next_rating_date_or_end_of_data
credit_rating_year
previous_fixed_quarter_date
days_since_call_on_fixed_quarter
days_since_rating
for_quarter
for_year
transcript
reportedCurrency
acceptedDate_balance_sheet
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory_balance_sheet
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayables
deferredRevenue
otherCurrentLiabilities
totalCurrentLiabilities
longTermDebt
deferredRevenueNonCurrent
deferredTaxLiabilitiesNonCurrent
otherNonCurrentLiabilities
totalNonCurrentLiabilities
otherLiabilities
capitalLeaseObligations
totalLiabilities
preferredSto

In [5]:
# Removing columns: 'Rating Rank AAA is 10', 'Investment_Grade', 'Change Direction Since Last Fixed Quarter Date', 'Change Since Last Fixed Quarter Date', 'Next Rating', 'Next Rating Date', 'next_rating_date_or_end_of_data'
df = df.drop(columns=['Rating Rank AAA is 10',  
                      'Investment_Grade', 
                      'Change Direction Since Last Fixed Quarter Date', 
                      'Change Since Last Fixed Quarter Date', 
                      'Next Rating', 
                      'Next Rating Date', 
                      'next_rating_date_or_end_of_data'])

In [6]:
# Get train df
train_df = df[df['train_test_80_20'] == 'train'].reset_index(drop=True)

## Fit AutoGluon

In [7]:
# Fit model
predictor = MultiModalPredictor(label='Rating').fit(train_df, save_path=os.path.expanduser('~/Box/STAT 222 Capstone/Autogluon/Autogluon_Multimodal_SCF'))

=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.11.8
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #96-Ubuntu SMP Wed Sep 20 08:23:49 UTC 2023
CPU Count:          96
Pytorch Version:    2.0.1+cu117
CUDA Version:       11.7
Memory Avail:       728.79 GB / 754.55 GB (96.6%)
Disk Space Avail:   53591.74 GB / 66962.99 GB (80.0%)


AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).


	10 unique label values:  ['BBB', 'BB', 'A', 'AA', 'B', 'CCC', 'AAA', 'CC', 'C', 'D']


	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])



AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /accounts/grad/ijyliu/Box/STAT 222 Capstone/Autogluon
    ```



...


Predicting DataLoader 0:  88%|████████▊ | 15/17 [00:06<00:00,  2.41it/s]


Predicting DataLoader 0:  94%|█████████▍| 16/17 [00:06<00:00,  2.53it/s]


Predicting DataLoader 0: 100%|██████████| 17/17 [00:06<00:00,  2.44it/s]


Predicting DataLoader 0: 100%|██████████| 17/17 [00:06<00:00,  2.44it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/accounts/grad/ijyliu/Box/STAT 222 Capstone/Autogluon")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).


